In [52]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy
from heapq import nlargest

In [53]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [54]:
# Load the English language model for SpaCy
nlp = spacy.load('en_core_web_sm')

In [55]:
# Original text
text = "The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."

In [56]:
# Tokenization
doc = nlp(text)
tokens = [token.text for token in doc]
print(tokens)

['The', 'Orbiter', 'Discovery', ',', 'OV-103', ',', 'is', 'considered', 'eligible', 'for', 'listing', 'in', 'the', 'National', 'Register', 'of', 'Historic', 'Places', '(', 'NRHP', ')', 'in', 'the', 'context', 'of', 'the', 'U.S.', 'Space', 'Shuttle', 'Program', '(', '1969', '-', '2011', ')', 'under', 'Criterion', 'A', 'in', 'the', 'areas', 'of', 'Space', 'Exploration', 'and', 'Transportation', 'and', 'under', 'Criterion', 'C', 'in', 'the', 'area', 'of', 'Engineering', '.', 'Because', 'it', 'has', 'achieved', 'significance', 'within', 'the', 'past', 'fifty', 'years', ',', 'Criteria', 'Consideration', 'G', 'applies', '.', 'Under', 'Criterion', 'A', ',', 'Discovery', 'is', 'significant', 'as', 'the', 'oldest', 'of', 'the', 'three', 'extant', 'orbiter', 'vehicles', 'constructed', 'for', 'the', 'Space', 'Shuttle', 'Program', '(', 'SSP', ')', ',', 'the', 'longest', 'running', 'American', 'space', 'program', 'to', 'date', ';', 'she', 'was', 'the', 'third', 'of', 'five', 'orbiters', 'built', 'b

In [57]:
tokens = word_tokenize(text)
sentences = sent_tokenize(text)
stop_words = set(stopwords.words('english'))

In [58]:
# Calculate word frequencies
sentences = sent_tokenize(text)
stop_words = set(stopwords.words('english'))
word_frequencies = {}

# Calculate word frequencies excluding stop words and punctuation
for word in nltk.word_tokenize(text):
    if word.lower() not in stop_words:
        if word.isalnum(): # Check if word consists of alphanumeric characters
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

In [59]:
# Normalize word frequencies
maximum_frequency = max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word] / maximum_frequency)

# Calculate sentence scores based on word frequencies
sentence_scores = {}
for sent in sentences:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30: # Consider only sentences with less than 30 words
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

In [60]:
# Select 30% of sentences with highest scores for summary
select_length = int(len(sentences)*0.3)  # вибираємо 30% речень для summary
summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
summary = ' '.join(summary)
print(summary)

Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew.


In [62]:
# Calculate sentence scores using SpaCy
doc = nlp(text)

sentence_scores = {}
for sent in doc.sents:
    sentence_word_count = len([token for token in sent if not token.is_punct])
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sentence_word_count < 30: # Consider only sentences with less than 30 words
                if sent.text not in sentence_scores.keys():
                    sentence_scores[sent.text] = word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent.text] += word_frequencies[word.text.lower()]

In [63]:
# Select 30% of sentences with highest scores for summary
select_length = int(len(list(doc.sents)) * 0.3)  # вибираємо 30% речень для summary
summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
summary = ' '.join(summary)
print(summary)

Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.


In [65]:
# Length of summary and original text
print(len(text))
print(len(summary))

2906
714


In [66]:
import pandas as pd

# Create DataFrame with original text and its summary
data = {'Input': [text], 'Output': [summary]}
df = pd.DataFrame(data)

# Set styles to align text to the center
df_styled = df.style.set_properties(**{'text-align': 'center'})

# Set styles to align column headers to the center
df_styled.set_table_styles([{'selector': 'th',
                             'props': [('text-align', 'center')]}])

# Display DataFrame with applied styles
display(df_styled)

,Input,Output
0,"The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound.","Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizer object
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Transform the text into a TF-IDF feature matrix
tfidf_matrix = tfidf_vectorizer.fit_transform([text])

# Get the list of all words in the matrix
words = tfidf_vectorizer.get_feature_names_out()

# Get the TF-IDF values for each word in the text
tfidf_values = tfidf_matrix.toarray()[0]

# Create a list of tuples (word, TF-IDF value) and sort it by TF-IDF value
keywords = [(words[i], tfidf_values[i]) for i in range(len(words))]
keywords.sort(key=lambda x: x[1], reverse=True)

# Print the top 10 keywords
print("Top 10 keywords:")
for keyword in keywords[:10]:
    print(keyword)

Top 10 keywords:
('space', 0.5095101710852533)
('shuttle', 0.3135447206678482)
('discovery', 0.2743516305843672)
('orbiter', 0.2743516305843672)
('criterion', 0.1567723603339241)
('missions', 0.1567723603339241)
('engineering', 0.11757927025044308)
('flew', 0.11757927025044308)
('fly', 0.11757927025044308)
('program', 0.11757927025044308)
